In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/My Drive/pneumonia_detection'

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/pneumonia_detection/models/research/:/content/gdrive/My Drive/pneumonia_detection/models/research/slim'

In [ ]:
# Always run the codes below for every session restart
!python setup.py build
!python setup.py install

In [ ]:
# Note: if you wish to know the remaining 
# hours you have for your colab session, 
# run the copy and run the code below.
import time, psutil
Start = time.time() - psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: {} hours'.format(Left/3600))

In [ ]:
%cd /content/gdrive/My Drive/pneumonia_detection/
os.getcwd()

# Prediction on Test Set

In [ ]:
from glob import glob
import cv2
import numpy as np
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

print(tf.__version__)

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/pneumonia_detection/models/research/object_detection⁩'

In [ ]:
# !echo $PYTHONPATH

In [ ]:
os.chdir('/content/gdrive/My Drive/pneumonia_detection/models/research/')
os.getcwd()

In [ ]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [ ]:
base_directory = '/content/gdrive/My Drive/pneumonia_detection/'
ckpt_directory = 'trainings/frcnn_resnet101_kitti_2018_01_28_512/frozen_graphs'
PATH_TO_CKPT = os.path.join(base_directory, ckpt_directory, '27954', 'frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(base_directory, 'data', 'label.pbtxt')

NUM_CLASSES = 1
os.path.exists(PATH_TO_CKPT), os.path.exists(PATH_TO_LABELS)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(categories)

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
TEST_IMAGE_PATHS_full = glob(os.path.join(base_directory, 'data', 'sample', '*.jpg'))

len(TEST_IMAGE_PATHS_full)

In [ ]:
TEST_IMAGE_PATHS_filtered = []
check = TEST_IMAGE_PATHS_full.copy()
# print(len(list(set(check))))
counter = 0

## removing duplicate images from list
for c in check:
  # print(c)
  if '(1)' in c:
    counter += 1
  else:
    TEST_IMAGE_PATHS_filtered.append(c)
print(len(TEST_IMAGE_PATHS_filtered), counter)

In [ ]:
TEST_IMAGE_PATHS = TEST_IMAGE_PATHS_filtered
# TEST_IMAGE_PATHS
IMAGE_SIZE = (12, 8)
len(TEST_IMAGE_PATHS)

In [ ]:
# TEST_IMAGE_PATHS = os.path.join(base_directory, 'data', 'test_jpgs', '12d70948-c153-45bd-9942-e6cc07d54d07.jpg')
# TEST_IMAGE_PATHS = TEST_IMAGE_PATHS_full[1]

In [ ]:
##############
## ORIGINAL ##
##############

results = {}

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
    # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for c, image_path in enumerate([TEST_IMAGE_PATHS]):
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            name = os.path.splitext(image_path.split('/')[-1])[0]
#             print('name', name)
            # print('boxes', boxes[0])
            # print('scores', scores)
            # print('classes', classes)
            # print('num', num)
            
            ## writing results
            results[name] = {'boxes': boxes,
                             'scores': scores,
                             'classes': classes,
                             'num': num}
            
            if c%500 == 0 and c>0:
              print(c)
            '''
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
            
            # image_name = image_path.split('/')[-1]
            # image_cv2 = image_np.copy()
            # image_cv2 = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
            # save_path = os.path.join(directory, 'test_images', 'test_resizer' '1303', image_name)
            # cv2.imwrite(save_path, image_cv2)
            
            plt.figure(figsize=IMAGE_SIZE)
            plt.imshow(image_np)
            # '''
# 0.21302113 0.10443303 0.6885848  0.31382525


In [ ]:
import pickle
with open(os.path.join(base_directory, 'results.pkl'), 'wb') as f:
  pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# creating image list with and without predictions

no_predictions = []
predictions = []

for key, values in results.items():
    
    num_det = int(values['num'][0])
    
    if num_det == 0:
        no_predictions.append(key)
    else:
        predictions.append(key)
        
len(no_predictions), len(predictions)

In [ ]:
import pandas as pd
df_no = pd.DataFrame(columns=['patientId', 'PredictionString'])
df_no['patientId'] = no_predictions
df_no.head()

In [ ]:
## checking scores of prediction and 
## appending for final inclusion

ids_list = []
pred_string_list = []
num_det_list = []
img_size = 1024

for i,image in enumerate(predictions):
    ids_list.append(image)

    image_result = results[image]
    num_det = int(image_result['num'][0])
    num_det_list.append(num_det)
    
    pred_string_list_int = []
    for j in range(num_det):
        score = image_result['scores'][0][j]

        if score>0.3:
        ymin = round(image_result['boxes'][0][j][0] * img_size)
        xmin = round(image_result['boxes'][0][j][1] * img_size)
        ymax = round(image_result['boxes'][0][j][2] * img_size)
        xmax = round(image_result['boxes'][0][j][3] * img_size)

        width = (xmax - xmin)
        height = (ymax - ymin)

        pred_string = (str(round(score, 2)) + ' ' +
                  str(int(xmin)) + ' ' +
                  str(int(ymin)) + ' ' +
                  str(int(width)) + ' ' +
                  str(int(height)))
        pred_string_list_int.append(pred_string)
    pred_string_list.append(' '.join(pred_string_list_int))
        
len(pred_string_list), len(ids_list)

In [ ]:
## creating dataframe with predictions

df_pred = pd.DataFrame(columns=['patientId', 'PredictionString'])
df_pred['patientId'] = ids_list
df_pred['PredictionString'] = pred_string_list
df_pred.head()

In [ ]:
## generating final csv for kaggle submission
df = pd.concat([df_pred, df_no])
df.shape

In [ ]:
os.getcwd()

In [ ]:
df.to_csv('/content/gdrive/My Drive/pneumonia_detection/output_score30%_v2.csv', sep=',', index=False)